In [ ]:
%load_ext autoreload
 
%autoreload 2

In [ ]:
from pfs.lam.imageAnalysis import neighbor_outlier_filter

from pfs.lam.detAnalysis import *
from pfs.lam.detFocusAnalysis import *
from pfs.lam.fileHandling import *
from pfs.lam.style import *
from pfs.lam.analysisPlot import *
import glob
import lsst.daf.persistence as dafPersist

In [ ]:
from pfs.lam.misc import find_nearest
import seaborn as sns


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime

In [ ]:
%matplotlib ipympl

In [ ]:
datetime.now().strftime("%Y-%m-%dT%Hh%M")

In [ ]:
roi_size = 24
doBck = True
threshold = 0.

In [ ]:
expIds = [13856,13857]
rerun = "sep2021"
cam = "r1"
extra = ''
basePath = '/drp/analysis/subaru/sm1/'
roi_size = 24


In [ ]:
expIds = [13857]
rerun = "sep2021"
cam = "r1"
extra = ''
basePath = '/drp/analysis/subaru/sm1/'
roi_size = 24


In [ ]:
expIds = [13866,13867]
rerun = "sep2021"
cam = "b1"
extra = ''
basePath = '/drp/analysis/subaru/sm1/'
roi_size = 24
Sources = ["neon","hgar"] 

In [ ]:
expIds = [13858,13859]
rerun = "sep2021"
cam = "b1"
extra = ''
basePath = '/drp/analysis/subaru/sm1/'
roi_size = 24
Sources = ["hgar", "neon"] 

In [ ]:
expIds = [13859]
rerun = "sep2021"
cam = "b1"
extra = ''
basePath = '/drp/analysis/subaru/sm1/'
roi_size = 24


# define defaut parameters


In [ ]:
#piston_index = "motor1"

com = True  # Center Of Mass
head = 0
tail = 0
criteria = 'EE5'
criterias = [criteria] #['EE5', 'sep_ECE5']
verbose = True
doPrint = False
arm = cam[0]
SM = f"SM{cam[1]}"
site = "Subaru"

In [ ]:
head = 0
tail = 0

In [ ]:
cluster = False

In [ ]:
doNeighborFilter = True
thres_neighbor = -0.1 # default -0.1

In [ ]:
doSave = False
doSaveCsv = False

# Set folder according place either cluster, laptop

In [ ]:
drpPath = "/data/drp/subaru/"
repo = "repo"
imgPath = basePath


# Read data CSV files for all experimentIds

In [ ]:
files = []
for experimentId in expIds:

    csvPath = imgPath+"Exp"+str(experimentId)+"/"+rerun+"_roi"+str(roi_size)+"/doBck"+str(doBck)+"/"+extra
    if not os.path.exists(csvPath):
        csvPath = imgPath+"Exp"+str(experimentId)+"/"+rerun+"/"+"roi"+str(roi_size)+"/doBck"+str(doBck)+"/"+extra
#    csvPath = imgPath+"Exp190/ws_slit_roi40/doBckTrue/"

    dataPath = csvPath
    print(dataPath)

    searchFile = f"{csvPath}Imquality_{cam}_Exp{experimentId}*"

    print(searchFile)
    files.extend(glob.glob(searchFile))

if verbose:
    print(*files, sep="\n")

imdata = pd.concat(map(pd.read_csv,files)).reset_index().sort_values(by="motor1")
#piston_imdata["wavelength"] = 300

#piston_imdata = piston_imdata.groupby(['wavelength','fiber', "motor1"]).mean().reset_index()

In [ ]:
#comment ="NoDarkFlat"
comment = f"{rerun}_roi{str(roi_size)}_doBck{str(doBck)}"
savePlotFile = csvPath+f"{SM}_{site}_{cam}_ExpIds_{'-'.join(str(x) for x in expIds)}_{'-'.join(str(x) for x in Sources)}"
savePlotFile = savePlotFile+f"_{comment}"
print(savePlotFile)
savePlotScatterFile = csvPath+f"{SM}_{site}_{cam}_ExpIds_{'-'.join(str(x) for x in expIds)}_{'-'.join(str(x) for x in Sources)}_scatter"
savePlotScatterFile = savePlotScatterFile+f"_{comment}"
print(savePlotScatterFile)
title = f"{SM} {site} {cam} ExpIds {' '.join(str(x) for x in expIds)} {' '.join(str(x) for x in Sources)}"
title = title + f"\n{comment}"
print(title)

# Data filtering

## Remove bad peak identified by SEP

In [ ]:
df = imdata[(imdata.sep_flag<1)]

## Remove peaks after a visual check (see at the end)

In [ ]:
waves = df.wavelength.unique()
print(waves)

In [ ]:
waves[9]

In [ ]:
####  Filtering of 
#
#expIds =  [13856,13857]
#rerun = "sep2021"
#cam = "r1"
#extra = ''
#basePath = '/drp/analysis/subaru/sm1/'
#roi_size = 24


#Sources = ["hgar", "neon"] #, "Neon"]

#because of saturation or CR
#waves filter were :846.5682 during SM1 first cooldown

#df = df[(df.EE5>0.8)] #=> checked saturation or CR
df = df[(df.EE5<1)]  #=> checked either no flux or double peak

ind_wave = 9
print(waves[ind_wave])
df = df[df.wavelength != waves[ind_wave]]



In [ ]:
####  Filtering of 
#
#expIds = [13866,13867]
#rerun = "sep2021"
#cam = "b1"
#extra = ''
#basePath = '/drp/analysis/subaru/sm1/'
#roi_size = 24


#Sources = ["hgar", "neon"] #, "Neon"]

#because of saturation or CR

df = df[(df.EE5>0.8)] #=> checked saturation or CR
df = df[(df.EE5<1)]  #=> checked saturation

In [ ]:
####  Filtering of 
#
#expIds =  [13858,13859]
#rerun = "sep2021"
#cam = "b1"
#extra = ''
#basePath = '/drp/analysis/subaru/sm1/'
#roi_size = 24


#Sources = ["hgar", "neon"] #, "Neon"]

#because of saturation or CR

df = df[(df.EE5>0.8)] #=> checked saturation or CR
df = df[(df.EE5<1)]  #=> checked saturation

# find the best EE5 value from the dither sequence

In [ ]:
best = []
for key,group in df.groupby(["wavelength","fiber"]):
#    best.append(group.where(group.EE5 == group.EE5.max()))
    best.append(group[group.EE5 == group.EE5.max()])
#    print(group.px.std())
#    print(group.py.std())

    #best.append(key)
best = pd.concat(best)

In [ ]:
best = best[best.EE5>0.5]

In [ ]:
len(best)

In [ ]:
doSavePlot = True


In [ ]:
plotImageQualityScatter(best, par="EE5", hist="EE5", savePlotFile=savePlotScatterFile+"best", title=title+" best", doSave=doSavePlot,\
                       vmin=0.89, vmax=1)

In [ ]:
plotImageQualityScatter(best, par="EE3", hist="EE3", savePlotFile=savePlotScatterFile+"best", title=title+" best", doSave=doSavePlot,\
                       vmin=0.45, vmax=0.73)

In [ ]:
plotImageQualityScatter(df, par="sep_brightness", savePlotFile=savePlotScatterFile+"best", title=title+" best", doSave=doSavePlot,\
                       vmin=-1, vmax=-1)

# checking bad EE5 to identify data that can be filtered
plot each peak to check what is the issue, CR, no flux, mutliple peaks ...

In [ ]:
pdf = df[(df.EE5<0.88)|(df.EE5>1)]
print(len(pdf))
print(pdf.wavelength.unique())
print(pdf.fiber.unique())
print(pdf.visit.unique())

In [ ]:
pdf = df[(df.EE5<0.5)]
print(len(pdf))
print(pdf.wavelength.unique())
print(pdf.fiber.unique())
print(pdf.visit.unique())

In [ ]:
pdf = best[(best.EE5<0.5)]
print(len(pdf))
print(pdf.wavelength.unique())
print(pdf.fiber.unique())
print(pdf.visit.unique())

In [ ]:
pdf = best[(best.EE5<0.83)]
print(len(pdf))

In [ ]:
pdf = best[(best.EE5>1)]
print(len(pdf))

In [ ]:
ind = 5

In [ ]:
visitId = int(pdf.visit.iloc[ind])
print(visitId)
print(pdf.wavelength.iloc[ind])
print(pdf.fiber.iloc[ind])
#visitId = 3449 

In [ ]:
import lsst.daf.persistence as dafPersist

In [ ]:
print(f"{drpPath}/{repo}/rerun/{rerun}")

In [ ]:
butler = dafPersist.Butler(f"{drpPath}/{repo}/rerun/{rerun}")
calExp = butler.get("calexp", visit=visitId, arm=arm)

In [ ]:
cx = pdf.px.iloc[ind]
cy = pdf.py.iloc[ind]
print(cx,cy)
#imagefile = imgPath+ pdf.file.iloc[ind]
#print(imagefile)

In [ ]:
plotOnePeak(calExp.image.array,cx, cy, roi_size=24, vmin=0, vmax= 10, doBck=False, verbose=True)

In [ ]:
range(len(pdf))

In [ ]:
print(f"{drpPath}/{repo}/rerun/{rerun}")
butler = dafPersist.Butler(f"{drpPath}/{repo}/rerun/{rerun}")


In [ ]:
plt.ion()

In [ ]:
for ind in range(len(pdf)):
    visitId = int(pdf.visit.iloc[ind])
    print(visitId)
    print(pdf.wavelength.iloc[ind])
    print(pdf.fiber.iloc[ind])
    calExp = butler.get("calexp", visit=visitId, arm=arm)
    cx = pdf.px.iloc[ind]
    cy = pdf.py.iloc[ind]
    print(cx,cy)
    plotOnePeak(calExp.image.array,cx, cy, roi_size=24, vmin=0, vmax= 10, doBck=False, verbose=True)
